# Terraform

With Terraform, you can describe your infrastructure using a special declarative language.

## CLI

To manipulate the terraform use special CLI. The basic commands are:

- `terraform init`: to init the terraform directory.
- `terraform validate`: to check if there are syntax mistakes in your terrform configuration.
- `terraform apply`: to apply the configuration.
- `terraform apply -destroy`: to unroll the infrastructure.

---

The following cell defines a simple Terraform configuration that deploys an `nginx` image in the Docker runtime.

In [22]:
rm -rf /tmp/terraform
mkdir /tmp/terraform
cat << EOF > /tmp/terraform/main.tf
terraform {
  required_providers {
    docker = {
      source = "kreuzwerker/docker"
      version = "~> 3.0.1"
    }
  }
}

provider "docker" {}

resource "docker_image" "nginx" {
  name         = "nginx:latest"
  keep_locally = false
}

resource "docker_container" "nginx" {
  image = docker_image.nginx.image_id
  name  = "terraform_example"
  ports {
    internal = 80
    external = 8000
  }
}
EOF
cd /tmp/terraform

The following cell show the terrafrom initialisation outputs:

In [23]:
terraform init

Initializing the backend...
Initializing provider plugins...
- Finding kreuzwerker/docker versions matching "~> 3.0.1"...
- Installing kreuzwerker/docker v3.0.2...
- Installed kreuzwerker/docker v3.0.2 (self-signed, key ID BD080C4571C6104C)
Partner and community providers are signed by their developers.
If you'd like to know more about provider signing, you can read about it here:
https://developer.hashicorp.com/terraform/cli/plugins/signing
Terraform has created a lock file .terraform.lock.hcl to record the provider
selections it made above. Include this file in your version control repository
so that Terraform can guarantee to make the same selections by default when
you run "terraform init" in the future.

Terraform has been successfully initialized!

You may now begin working with Terraform. Try running "terraform plan" to see
any changes that are required for your infrastructure. All Terraform commands
should now work.

If you ever set or change modules or backend configuration fo

Special folders are created in the terraform directory:

In [24]:
ls -la

total 56
drwxrwxr-x  3 user user  4096 paź 28 15:53 .
drwxrwxrwt 25 root root 36864 paź 28 15:53 ..
-rw-rw-r--  1 user user   400 paź 28 15:53 main.tf
drwxr-xr-x  3 user user  4096 paź 28 15:53 .terraform
-rw-r--r--  1 user user  1337 paź 28 15:53 .terraform.lock.hcl


The validation:

In [14]:
terraform validate

Success! The configuration is valid.



The following code applies the configuration. The `-auto-approve` flag removes the terraform prompts to confirm that you want to apply the changes.

In [17]:
terraform apply -auto-approve &> /dev/null

The specified `nginx` container appeared in the docker runtime.

In [18]:
docker ps

CONTAINER ID   IMAGE          COMMAND                  CREATED        STATUS        PORTS                  NAMES
e98472d16175   657fdcd1c365   "/docker-entrypoint.…"   1 second ago   Up 1 second   0.0.0.0:8000->80/tcp   terraform_example


The following cell shows that after `terraform apply -destroy` the container deployed by terraform disappered:

In [25]:
terraform apply -auto-approve -destroy &> /dev/null
docker ps

CONTAINER ID   IMAGE     COMMAND   CREATED   STATUS    PORTS     NAMES
